In [1]:
import os
import boto3
import uuid
import mlflow
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
# data parameters
year = 2021
month = 1
taxi_type = 'green'

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}_tripdata_predictions_{year:04d}-{month:02}.parquet'

# AWS backend store parameters
RUN_ID = os.getenv('RUN_ID', '76d5937ab42549dfbc4f86b58bc7f71d')

In [3]:
# Initialize a boto3 session with specific AWS credentials
session = boto3.Session(
    aws_access_key_id='',
    aws_secret_access_key='',
    region_name=''
)

# Set the boto3 session as the default for MLflow
boto3.setup_default_session(
    aws_access_key_id='',
    aws_secret_access_key='',
    region_name=''
)


In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df['ride_id'] = generate_uuids(len(df))
    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

In [5]:
def load_model(run_id):
    logged_model = f's3://mlflow-artifacts-remote-pythondance/1/{RUN_ID}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model

def apply_model(input_file, run_id, output_file):
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration'] 
    df_result['model_version'] = RUN_ID
    df_result.to_parquet(output_file, index=False)
    return df_result

In [6]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff,model_version
0,7714495d-172d-42f0-a637-bd9179c43ef4,2021-01-01 00:15:56,43,151,3.933333,6.862711,-2.929378,76d5937ab42549dfbc4f86b58bc7f71d
1,399b4173-24ae-493c-bf46-02b6157b675e,2021-01-01 00:25:59,166,239,8.750000,13.368721,-4.618721,76d5937ab42549dfbc4f86b58bc7f71d
2,f186e5ae-f9c7-417a-a2b3-7094099f373f,2021-01-01 00:45:57,41,42,5.966667,6.360871,-0.394204,76d5937ab42549dfbc4f86b58bc7f71d
3,beef683d-d036-419e-b95a-e36bcf7c6741,2020-12-31 23:57:51,168,75,7.083333,11.824423,-4.741089,76d5937ab42549dfbc4f86b58bc7f71d
7,75f66a66-7d74-4e2e-83f3-6388e4e799ce,2021-01-01 00:26:31,75,75,2.316667,3.389290,-1.072623,76d5937ab42549dfbc4f86b58bc7f71d
...,...,...,...,...,...,...,...,...
76513,6ff2ac5b-d924-4670-98ae-1567b89df5d9,2021-01-31 21:38:00,81,90,38.000000,41.526829,-3.526829,76d5937ab42549dfbc4f86b58bc7f71d
76514,f212c871-92f2-4ce9-84d6-b10f95ab8830,2021-01-31 22:43:00,35,213,38.000000,43.858974,-5.858974,76d5937ab42549dfbc4f86b58bc7f71d
76515,c50a88f2-d729-4fab-8cfb-0bb0afed5929,2021-01-31 22:16:00,74,69,11.000000,14.436509,-3.436509,76d5937ab42549dfbc4f86b58bc7f71d
76516,d4ede608-c7d5-4b47-8ead-68a45499b49f,2021-01-31 23:10:00,168,215,27.000000,37.092622,-10.092622,76d5937ab42549dfbc4f86b58bc7f71d


In [8]:
!ls output/

green_tripdata_predictions_2021-01.parquet


In [9]:
# create new directory
# !mkdir output